<a href="https://colab.research.google.com/github/KangHyeonGu1/-/blob/main/solar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv("solar data.txt")
data.head

<bound method NDFrame.head of           temp  cloud cover      wind  humidity  barometer  day_power
0     0.312052     0.142857  0.234784  0.778769   0.638520   0.058824
1     0.369939     0.285714  0.412924  0.725910   0.424439   0.058824
2     0.352761     0.571429  0.572187  0.709644   0.296313   0.058824
3     0.348520     0.285714  0.334605  0.751113   0.118489   0.029412
4     0.348466     0.714286  0.313384  0.735478   0.160241   0.029412
...        ...          ...       ...       ...        ...        ...
1327  0.263113     0.714286  0.268356  0.786389   0.318577   0.058824
1328  0.252461     0.714286  0.105177  0.762446   0.423438   0.117647
1329  0.257106     0.142857  0.103314  0.809187   0.474831   0.088235
1330  0.326789     0.857143  0.292398  0.759844   0.455705   0.000000
1331  0.287142     0.428571  0.232619  0.681145   0.565277   0.176471

[1332 rows x 6 columns]>

In [3]:
# 독립변수 기온 spread설정과 삼각퍼지화
te=data['temp']
spread= [None] * 1332
for i in range(0,1331):
  spread[i]= (np.abs(te[i+1]-te[i]))/2
spread[1331]=spread[1330]
x1= te-spread
x2= te+spread
xdf=pd.DataFrame({'a':x1,'b':te,'c':x2})
xa=xdf[xdf['a']< 0]
xdf.loc[xa.index,'a']=0
x=xdf.to_numpy()


In [4]:
# 매개변수 습도 spread설정과 삼각퍼지화. 0-1사이로 정규화 하였기 때문에
# 일단 기온과 같이 spread화 했지만 생각할 필요성이 있음. 
hu=data['humidity']
spreadm= [None] * 1332
for i in range(0,1331):
  spreadm[i]= (np.abs(hu[i+1]-hu[i]))/2
spreadm[1331]=spreadm[1330]
m1= hu-spreadm
m2= hu+spreadm
mdf=pd.DataFrame({'a':m1,'b':hu,'c':m2})
ma=mdf[mdf['a']< 0]
mdf.loc[ma.index,'a']=0
m=mdf.to_numpy()

In [5]:
# 종속변수
power=data['day_power']
ydf=pd.DataFrame({'a':power,'b':power,'c':power})
y= ydf.to_numpy()


In [6]:
# 조절변수1
cl=data['cloud cover']
w1= cl-0.5
w2= cl+0.5
wdf=pd.DataFrame({'a':w1,'b':cl,'c':w2})
wa=wdf[wdf['a']< 0]
wdf.loc[wa.index,'a']=0
w=wdf.to_numpy()


In [7]:
# 조절변수2
wi=data['wind']
spreadz= [None] * 1332
for i in range(0,1331):
  spreadz[i]= (np.abs(wi[i+1]-wi[i]))/2
spreadz[1331]=spreadz[1330]
z1= wi-spreadz
z2= wi+spreadz
zdf=pd.DataFrame({'a':z1,'b':wi,'c':z2})
za=zdf[zdf['a']< 0]
zdf.loc[za.index,'a']=0
z=zdf.to_numpy()

In [8]:
# 
xw=x*w
xz=x*z
wz=w*z
xwz=x*w*z

In [12]:
# m= b0 + b1x + b2w + b3z + b4xw + b5xz + b6wz + b7xwz
def dc(a,b): # 삼각퍼지 행렬곱 함수
  c=a*b
  sum=c.sum()
  return sum
ones=np.ones(shape=(1332,3))
xtx=np.zeros(shape=(8,8))
xlist=[ones,x,w,z,xw,xz,wz,xwz]
for i in range(0,8):   # 회귀계수 추정을 위한 삼각퍼지 x대치행렬과 x행렬 행렬곱 for문
  for j in range(0,8):
    xtx[i,j]= dc(xlist[i],xlist[j])
xtm=np.zeros(shape=(8,1))   
for i in range(0,8):   # 회귀계수 추정을 위한 삼각퍼지 x대치행렬과 y행렬 행렬곱 for문
  for j in range(0,1):
    xtm[i,j]= dc(xlist[i],m)
lxtx=np.linalg.inv(xtx)
b=lxtx@xtm
b=np.around(b, decimals=4)
b

array([[ 0.8603],
       [-0.6239],
       [ 0.1469],
       [-0.4784],
       [ 0.0088],
       [ 1.0042],
       [ 0.177 ],
       [-0.4559]])

M= 0.6245 - 0.3944X + 0.1872W -0.0195Z -0.1291XW + 0.6734XZ - 0.1942WZ - 2.2929XWZ (crisp data)

M= 0.8603 - 0.6239X + 0.1469W - 0.4784Z + 0.0088XW + 1.0042XZ + 0.177WZ - 0.4559XWZ (삼각퍼지화 data)


In [13]:
xtx2=np.zeros(shape=(3,3))
xlist2=[ones,x,m]
for i in range(0,3):   # 회귀계수 추정을 위한 삼각퍼지 x대치행렬과 x행렬 행렬곱 for문
  for j in range(0,3):
    xtx2[i,j]= dc(xlist2[i],xlist2[j])
xty=np.zeros(shape=(3,1))   
for i in range(0,3):   # 회귀계수 추정을 위한 삼각퍼지 x대치행렬과 y행렬 행렬곱 for문
  for j in range(0,1):
    xty[i,j]= dc(xlist2[i],y)
lxtx2=np.linalg.inv(xtx2)
b2=lxtx2@xty
b2=np.around(b2, decimals=4)
b2

array([[ 0.5432],
       [ 0.5348],
       [-0.7551]])

Y= 0.8542 + 0.4950X - 0.8307M (crisp data)

Y= 0.5432 + 0.5348X - 0.7551M (삼각퍼지화 데이터)